<a href="https://colab.research.google.com/github/sayakpaul/SimSiam-TF/blob/main/SimSiam_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook evaluates the frozen features of a ResNet50 pre-trained (50 epochs of pre-training) using the [SimSiam method](https://arxiv.org/abs/2011.10566). You can refer to the [pre-training notebook here](https://github.com/sayakpaul/SimSiam-TF/blob/main/SimSiam_Pre_training.ipynb). Following evaluation methods have been used - 

* Linear evaluation of the frozen features

*Please note that to keep this minimal I did not follow the original hyperparameter configurations rigorously.*  

## Initial setup

In [1]:
!nvidia-smi

Mon Dec  6 22:01:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 107...  Off  | 00000000:01:00.0  On |                  N/A |
| 12%   52C    P8    15W / 180W |    397MiB /  8111MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [3]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import matplotlib.pyplot as plt
import numpy as np
import random
import time
import os

tf.random.set_seed(666)
np.random.seed(666)

In [4]:
# Retrieve the weights (50 epochs pre-training) of the projection head
#!wget -q https://github.com/sayakpaul/SimSiam-TF/releases/download/v1.0.0/projection.h5

## Dataset collection & preparation

The following code is referred from [here](https://github.com/ayulockin/SwAV-TF/blob/master/linear_evaluation/Linear_Evaluation_10_Epochs.ipynb). 

In [5]:
# Gather Flowers dataset
train_ds, validation_ds = tfds.load(
    "imagenet2012_subset/10pct",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True
)

AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

@tf.function
def scale_resize_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (224, 224)) # Resizing to highest resolution used while training swav
    return (image, label)

training_ds = (
    train_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

validation_ds = (
    validation_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

2021-12-06 22:01:07.551713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 22:01:07.594203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 22:01:07.594610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 22:01:07.596003: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## Utility functions

In [6]:
def get_encoder():
    base_model = tf.keras.applications.ResNet50(include_top=False,
        weights=None, input_shape=(224, 224, 3))
    base_model.trainable = True

    inputs = tf.keras.layers.Input((224, 224, 3))
    x = base_model(inputs, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(2048, activation='relu', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    z = tf.keras.layers.Dense(2048)(x)

    f = tf.keras.Model(inputs, z)

    return f

In [7]:
get_encoder().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2048)              4194304   
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 2048)              419635

In [8]:
# We now load up the pre-trained weights
projection = get_encoder()
projection.load_weights('projection_022.h5')

# Create a sub-model for extracting features
rn50 = tf.keras.Model(projection.input, projection.layers[2].output)
rn50.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
def get_linear_classifier(feature_backbone, trainable=False):
    inputs = tf.keras.layers.Input(shape=(224, 224, 3))
    
    feature_backbone.trainable = trainable
    x = feature_backbone(inputs, training=False)
    outputs = tf.keras.layers.Dense(1000, activation="softmax", )(x)
    linear_model = tf.keras.Model(inputs, outputs)

    return linear_model

In [10]:
get_linear_classifier(rn50).summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 model_2 (Functional)        (None, 2048)              23587712  
                                                                 
 dense_4 (Dense)             (None, 1000)              2049000   
                                                                 
Total params: 25,636,712
Trainable params: 2,049,000
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
def plot_progress(hist):
    plt.plot(hist.history["loss"], label="train_loss")
    plt.plot(hist.history["val_loss"], label="validation_loss")
    plt.plot(hist.history["accuracy"], label="training_accuracy")
    plt.plot(hist.history["val_accuracy"], label="validation_accuracy")
    plt.title("Training Progress")
    plt.ylabel("accuracy/loss")
    plt.xlabel("epoch")
    plt.legend(loc="upper left")
    plt.grid()
    plt.show()

## Linear evaluation

In [12]:
# Early Stopping to prevent overfitting
#early_stopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
#                                                 patience=5, verbose=2, 
#                                                 restore_best_weights=True)

In [13]:
# Get linear model and compile
tf.keras.backend.clear_session()
model = get_linear_classifier(rn50)
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"],
                     optimizer="adam")

# Train 
history = model.fit(training_ds,
                 validation_data=validation_ds,
                 epochs=50,
                 #callbacks=[early_stopper]
                 )
plot_progress(history)

Epoch 1/50


2021-12-06 22:01:13.865540: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2021-12-06 22:01:14.475824: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-12-06 22:01:14.476505: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-12-06 22:01:14.476522: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2021-12-06 22:01:14.477145: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-12-06 22:01:14.477186: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1702/1702 [==============================] - 347s 202ms/step - loss: 9.7715 - accuracy: 0.0018 - val_loss: 10.0044 - val_accuracy: 0.0020
Epoch 2/50
1702/1702 [==============================] - 343s 202ms/step - loss: 9.4492 - accuracy: 0.0034 - val_loss: 9.9697 - val_accuracy: 0.0022
Epoch 3/50
1702/1702 [==============================] - 332s 195ms/step - loss: 9.2100 - accuracy: 0.0052 - val_loss: 9.9318 - val_accuracy: 0.0031
Epoch 4/50
1702/1702 [==============================] - 310s 182ms/step - loss: 9.0636 - accuracy: 0.0071 - val_loss: 9.8862 - val_accuracy: 0.0032
Epoch 5/50
1702/1702 [==============================] - 310s 182ms/step - loss: 8.8602 - accuracy: 0.0095 - val_loss: 9.9023 - val_accuracy: 0.0040
Epoch 6/50
1702/1702 [==============================] - 310s 182ms/step - loss: 8.7196 - accuracy: 0.0120 - val_loss: 9.9537 - val_accuracy: 0.0036
Epoch 7/50
1702/1702 [==============================] - 310s 182ms/step - loss: 8.5531 - accuracy: 0.0152 - val_loss: 9.96

KeyboardInterrupt: 

In [ ]:
_, acc = model.evaluate(validation_ds)
print('Validation accuracy:', round(acc*100, 2))

For a minimal implementation the scores aren't that bad. 

In [ ]:
def plot_progress_acc(hist):
    fig, axs = plt.subplots(1,1, figsize=(10,6))
    plt.plot(hist.history["accuracy"], label="training_accuracy")
    plt.plot(hist.history["val_accuracy"], label="validation_accuracy")
    plt.title("Training Progress -- Accuracy", size=20)
    plt.ylabel("accuracy", size=20); plt.yticks(size=15)
    plt.xlabel("epoch", size=20); plt.xticks(size=15)
    plt.legend(loc="upper left")
    plt.grid()
    plt.show()

def plot_progress_loss(hist):
    fig, axs = plt.subplots(1,1, figsize=(10,6))
    plt.plot(hist.history["loss"], label="train_loss")
    plt.plot(hist.history["val_loss"], label="validation_loss")
    plt.title("Training Progress -- Loss", size=20)
    plt.ylabel("loss", size=20); plt.yticks(size=15)
    plt.xlabel("epoch", size=20); plt.xticks(size=15)
    plt.legend(loc="lower left")
    plt.grid()
    plt.show()

plot_progress_acc(history)
plot_progress_loss(history)